In [1]:
# TTC Streetcar Delay Data - Combine CSV & Excel (All Sheets)
# Reads both the CSV and Excel files (all sheets inside the Excel file)
# Standardizes the column names and schema
# Merges both into one consistent dataset
# Combines bound and code into a single code column
# Saves the result as a single clean .csv file, ready for modeling (feature engineering & training)

import pandas as pd
import glob, os

DATA_PATH = "data"

# Read CSV (2025)
csv_file = os.path.join(DATA_PATH, "TTC Streetcar Delay Data since 2025.csv")
df_csv = pd.read_csv(csv_file)
df_csv.columns = df_csv.columns.str.strip().str.lower().str.replace(" ", "_")
df_csv = df_csv.loc[:, ~df_csv.columns.duplicated()]  # ensure unique headers
df_csv.drop(columns=["_id"], inplace=True, errors="ignore")

print(f"Loaded 1 sheets from ttc-streetcar-delay-data-2025")

# Read Excel (2014–2024, all sheets)
xlsx_files = glob.glob(os.path.join(DATA_PATH, "*.xlsx"))
frames = []
for file in xlsx_files:
    try:
        sheets = pd.read_excel(file, sheet_name=None, engine="openpyxl")
        for name, sheet in sheets.items():
            sheet["source_file"] = os.path.basename(file)
            sheet["month"] = name
            frames.append(sheet)
        print(f" Loaded {len(sheets)} sheets from {os.path.basename(file)}")
    except Exception as e:
        print(f" Skipping {file}: {e}")

df_xlsx = pd.concat(frames, ignore_index=True)
df_xlsx.columns = df_xlsx.columns.str.strip().str.lower().str.replace(" ", "_")
df_xlsx = df_xlsx.loc[:, ~df_xlsx.columns.duplicated()]  # ensure unique headers

# Rename Excel columns to match CSV schema
rename_map = {
    "report_date": "date",
    "route": "line",
    "location": "station",
    "incident": "code",
    "direction": "bound",
    "min_delay": "min_delay",
    "min_gap": "min_gap"
}
df_xlsx.rename(columns=rename_map, inplace=True)

# Define target columns (shared structure)
common_cols = [
    "date", "line", "time", "day", "station", "code",
    "min_delay", "min_gap", "bound", "vehicle"
]

# Safely align both datasets to available columns
df_xlsx = df_xlsx[[c for c in common_cols if c in df_xlsx.columns]]
df_csv = df_csv[[c for c in common_cols if c in df_csv.columns]]

# Guarantee unique columns before concat
df_xlsx = df_xlsx.loc[:, ~df_xlsx.columns.duplicated()]
df_csv = df_csv.loc[:, ~df_csv.columns.duplicated()]

# Combine both datasets
df_all = pd.concat([df_xlsx, df_csv], ignore_index=True, sort=False)


# Basic cleanup and type conversion

# Convert date
df_all["date"] = pd.to_datetime(df_all["date"], errors="coerce")

# Fix mixed time formats (12-hour and 24-hour)
def parse_time(value):
    if pd.isna(value):
        return None
    str_val = str(value).strip()
    for fmt in ("%I:%M:%S %p", "%I:%M %p", "%H:%M:%S", "%H:%M"):
        try:
            return pd.to_datetime(str_val, format=fmt).strftime("%H:%M")
        except Exception:
            continue
    return None

df_all["time"] = df_all["time"].apply(parse_time)

# Extract hour as numeric feature for modeling
df_all["hour"] = pd.to_datetime(df_all["time"], format="%H:%M", errors="coerce").dt.hour

# Clean numeric columns
df_all["min_delay"] = pd.to_numeric(df_all["min_delay"], errors="coerce").fillna(0)
df_all["min_gap"] = pd.to_numeric(df_all["min_gap"], errors="coerce").fillna(0)

# Drop empty critical fields
df_all.dropna(subset=["line", "code", "station"], inplace=True)

# Save to clean CSV
output_file = os.path.join(DATA_PATH, "TTC_Streetcar_Delays_2014_2025.csv")
df_all.to_csv(output_file, index=False)
print(f"\n Saved combined file to: {output_file}")

Loaded 1 sheets from ttc-streetcar-delay-data-2025
 Loaded 12 sheets from ttc-streetcar-delay-data-2014.xlsx
 Loaded 12 sheets from ttc-streetcar-delay-data-2015.xlsx
 Loaded 12 sheets from ttc-streetcar-delay-data-2016.xlsx
 Loaded 12 sheets from ttc-streetcar-delay-data-2017.xlsx
 Loaded 12 sheets from ttc-streetcar-delay-data-2018.xlsx
 Loaded 12 sheets from ttc-streetcar-delay-data-2019.xlsx
 Loaded 12 sheets from ttc-streetcar-delay-data-2020.xlsx
 Loaded 12 sheets from ttc-streetcar-delay-data-2021.xlsx
 Loaded 1 sheets from ttc-streetcar-delay-data-2022.xlsx
 Loaded 1 sheets from ttc-streetcar-delay-data-2023.xlsx
 Loaded 1 sheets from ttc-streetcar-delay-data-2024.xlsx

 Saved combined file to: data\TTC_Streetcar_Delays_2014_2025.csv
